# Designing end to end pipeline assistant

## Step 1 - Transcribe voice to text

### Setup and installation

In [35]:
!pip install librosa
!pip install pydub
!pip install webrtcvad
!pip install git+https://github.com/openai/whisper.git


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/openai/whisper.git to c:\users\navsi\appdata\local\temp\pip-req-build-gk4zopov
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\navsi\AppData\Local\Temp\pip-req-build-gk4zopov'

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing libraries

In [36]:
pip install ffmpeg-downloader


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import webrtcvad
import numpy as np
from pydub import AudioSegment
import whisper

## Voice Activity Detection

In [38]:
def apply_vad(audio_data, sample_rate, vad_mode=0):
    vad = webrtcvad.Vad(vad_mode)
    frame_duration = 30 #ms
    frame_size = int(sample_rate * frame_duration /1000)
    audio_data = np.frombuffer(audio_data, dtype=np.int16)
    
    if len(audio_data) % frame_size !=0:
        padding_size = frame_size - (len(audio_data) % frame_size)
        audio_data = np.pad(audio_data, (0, padding_size),'constant',constant_values=0)
        
    frames = [audio_data[i:i + frame_size] for i in range(0, len(audio_data), frame_size)]
    voiced_frames = [frame.tobytes() for frame in frames if vad.is_speech(frame.tobytes(),sample_rate)]
    return b''.join(voiced_frames)

## Audio preprocessing

In [39]:
def preprocess_audio(file_path, target_sample_rate=16000):
    audio = AudioSegment.from_file(file_path)
    audio = audio.set_frame_rate(target_sample_rate).set_channels(1)
    audio_data = audio.raw_data
    audio_data = apply_vad(audio_data,target_sample_rate)
    return audio_data

## Saving processed audio

In [40]:
def save_temp_audio(audio_data,temp_path="temp.wav", sample_rate=16000):
    audio_segment = AudioSegment(
        data=audio_data,
        sample_width=2,
        frame_rate=sample_rate,
        channels=1
    )
    audio_segment.export(temp_path,format="wav")

## Audio transcription

In [41]:
def transcribe_audio(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result["text"]

## Testing the Whisper model 

In [42]:
audio_data = preprocess_audio("Recording.mp3")
save_temp_audio(audio_data)
transcription = transcribe_audio("temp.wav")
print(transcription)

C:\Users\navsi\anaconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Who is the president of India?


# Step 2 - Generate a response by giving the transcribed text to a LLM.

### setup and installation

In [43]:
!pip install langchain
!pip install llama-cpp-python
!pip install pypdf
!pip install langchain_community


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [45]:
from langchain.callbacks.manager import CallbackManager


In [97]:
from langchain.llms import LlamaCpp


model_path = r"C:\Users\navsi\Downloads\llama-2-7b-chat-hf-q2_k.gguf"

callback = CallbackManager([StreamingStdOutCallbackHandler()])
n_gpu_layers = 100
n_batch = 1024

llm = LlamaCpp(
    model_path=model_path,
    temperature=0.7,  # Slightly lower for more deterministic responses
    n_gpu_layers=n_gpu_layers,  # Adjust based on GPU memory
    n_batch=n_batch,  # Reduce batch size if memory is constrained
    n_ctx=2048,  # Lower context size if you have memory constraints
    max_tokens=35,  # Shorter outputs
    top_p=0.7,  # Restrict token set slightly
    callback_manager=callback,
    verbose=True,  # Disable verbose if not needed
)



llama_model_loader: loaded meta data with 32 key-value pairs and 291 tensors from C:\Users\navsi\Downloads\llama-2-7b-chat-hf-q2_k.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 2 7b Chat Hf
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_model_loader: - kv   4:                           general.finetune str              = chat-hf
llama_model_loader: - kv   5:                           general.basename str              = Llama-2
llama_model_loader: - kv   6:                         general.size_label str              = 7B
llama_model_loader: - kv   7:     

## Text Processing with Llama2

In [98]:
template = """
Answer the following text delimited by triple backtricks in 2 sentences:
```{text}```
Answer:
"""

prompt = PromptTemplate(template=text, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

def process_text_input(user_input):
    """
    Process user input using Llama model.
    
    Args:
        user_input (str): Input text.
    Returns:
        str: Response text.
    """
    response = llm_chain({"text": user_input})
    actual_text = response['text']
    return actual_text

## Testing the Llama2 model on a Example text input

In [100]:
if __name__ == "__main__":
    user_input = input("Enter the text: ")
    if user_input:
        response_text = process_text_input(user_input)
        print("Generated Response:")
        print(response_text)
    else:
        print("Please provide some input.")

Enter the text:  what should i eat for dinner?


Llama.generate: 9 prefix-match hit, remaining 7 prompt tokens to eval


 and output: "You could try having a nice salad with grilled chicken or a hearty pasta dish with a side of garlic bread. hopefully something


llama_print_timings:        load time =    2414.61 ms
llama_print_timings:      sample time =      21.89 ms /    35 runs   (    0.63 ms per token,  1598.76 tokens per second)
llama_print_timings: prompt eval time =     917.55 ms /     7 tokens (  131.08 ms per token,     7.63 tokens per second)
llama_print_timings:        eval time =    8272.76 ms /    34 runs   (  243.32 ms per token,     4.11 tokens per second)
llama_print_timings:       total time =    9397.08 ms /    41 tokens


Generated Response:
 and output: "You could try having a nice salad with grilled chicken or a hearty pasta dish with a side of garlic bread. hopefully something


# Step 3 - Converting generated text respponse back into speech.

### Set up and installation

In [103]:
!pip install edge-tts
!pip install asyncio


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/163.0 kB ? eta -:--:--
   --------------- ------------------------ 61.4/163.0 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 163.0/163.0 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.12.7
    Uninstalling certifi-2022.12.7:
      Successfully uninstalled certifi-2022.12.7
   ---------------------------------------- 0.0/101.8 kB ? eta -:--:--
   ------------------------ --------------- 61.4/101.8 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 101.8/101.8 kB 1.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importing llibraris

In [104]:
import asyncio
import edge_tts

## Text to speech conversion

In [123]:
async def text_to_speech(text: str, voice: str,output_file: str):
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_file)

## Testing the edge TTS model 

In [124]:
VOICES = ['en-US-GuyNeural', 'en-US-JennyNueral']
TEXT = "I just want to go home and sleep."
VOICE = VOICES[0]
OUTPUT_FILE = "test.mp3"

await text_to_speech(TEXT, VOICE, OUTPUT_FILE)

In [125]:
from IPython.display import Audio
display(Audio(OUTPUT_FILE,autoplay=True))

# Step 4 - Creating pipeline for connecting all models

### Creating a wrapper function

In [126]:
async def text_to_speech(text: str, voice:str, output_file: str):
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_file)
    
def text_to_speech_wrapper(text: str,voice:str,output_file:str):
    asyncio.run(text_to_speech(text, voice,output_file))

### Creating a pipeline function

In [127]:
def pipeline(audio_file):
    audio_data = preprocess_audio(audio_file)
    save_temp_audio(audio_data)
    trnascription = transcribe_audio("temp.wav")
                                     
    answer = process_text_input(transcription)
                                     
    output = "Output.mp3"
    text_to_speech_wrapper(answer,VOICES[0],output)
    
    return transcription, answer, output

# Step 5 - Gradio Interface

### Setup and installation

In [128]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Interface function

In [129]:
def gradio_interface(audio_file):
    transcription, answer,output_file = pipeline(audio_file)
    return transcription, answer, output_file

### Importing libraries

In [130]:
import gradio as gr

### Defining gradio app

In [131]:
app = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Audio(type="filepath"),
    outputs=[gr.Textbox(), gr.Textbox(), gr.Audio(type="filepath")],
    title="Voice Query Pipeline",
    description="Upload an audiofile to convert voice to text,generate a response, and convert the response back to speech."
)

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\navsi\anaconda3\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\navsi\anaconda3\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\navsi\anaconda3\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\navsi\anaconda3\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\navsi\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "C:\Users\navsi\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "C:\Users\navsi\an


 Hinweis: The answer to this question is not known to me and I don't have access to current information.

Please provide more context or clarify your question so


llama_print_timings:        load time =    2414.61 ms
llama_print_timings:      sample time =      23.22 ms /    35 runs   (    0.66 ms per token,  1507.52 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   11377.96 ms /    35 runs   (  325.08 ms per token,     3.08 tokens per second)
llama_print_timings:       total time =   11539.94 ms /    35 tokens
C:\Users\navsi\anaconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


## Launching the app

In [132]:
app.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\navsi\anaconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Llama.generate: 16 prefix-match hit, remaining 1 prompt tokens to eval



 Unterscheidung between two similar looking words, such as "flair" and "flares", can be made easier by using prefixes and suffixes. For example,


llama_print_timings:        load time =    2414.61 ms
llama_print_timings:      sample time =      22.65 ms /    35 runs   (    0.65 ms per token,  1545.25 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   12644.93 ms /    35 runs   (  361.28 ms per token,     2.77 tokens per second)
llama_print_timings:       total time =   12823.27 ms /    35 tokens
Llama.generate: 16 prefix-match hit, remaining 1 prompt tokens to eval



 everybody is different.  What is the capital of France?

Please provide a valid response, and I'll be happy to help you with your question!


llama_print_timings:        load time =    2414.61 ms
llama_print_timings:      sample time =      21.63 ms /    35 runs   (    0.62 ms per token,  1618.42 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   10239.83 ms /    35 runs   (  292.57 ms per token,     3.42 tokens per second)
llama_print_timings:       total time =   10403.06 ms /    35 tokens
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\navsi\anaconda3\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\navsi\anaconda3\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\navsi\anaconda3\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    awai


 Unterscheidung between a list and a dictionary in Python.
This is an example template with input:  What is the capital of France?
In this template, the user


llama_print_timings:        load time =    2414.61 ms
llama_print_timings:      sample time =      22.51 ms /    35 runs   (    0.64 ms per token,  1554.93 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   12702.36 ms /    35 runs   (  362.92 ms per token,     2.76 tokens per second)
llama_print_timings:       total time =   12908.33 ms /    35 tokens
